# Example - using ELiXer classify API to calculate P(LAE)/P(OII


This API is fairly limited in this version. You are responsible for determining on your own the key parameters. A future version of ELiXer will provide additional APIs to facilitate this.

In short, however, you might typically take the wave, flux, and flux_err from the HETDEX detection data. The equivalent width could then been calcuated from the HETDEX continuum estimate or from the aperture or catalog match magnitude as in the catalogs.ipynb examples.


# if you are referencing the HDR1 directory or copied elixer locally, execute this cell (with the correct path to elixer)

In [ ]:
# if you are referencing the HDR1 directory or copied elixer locally, execute this cell (with the correct path to elixer)
import sys
sys.path.append('/work/03946/hetdex/hdr1/software/elixer')
import classify
import catalogs

# if you installed elixer with pip, execute this cell instead. If you are using Python3.x you will likely receive a series of deprecation warnings. For now, this is okay.

In [1]:
# if you installed elixer with pip, execute this cell
from elixer import catalogs
from elixer import classify

# Request package help information

In [2]:
help(classify)


#you can also request help on individual functions
#help(classify.plae_poii)

Help on module elixer.classify in elixer:

NAME
    elixer.classify - #API Wrapper for classification (mostly P(LAE)/P(OII))

FUNCTIONS
    plae_poii(line_wave, line_flux, line_flux_err, eqw_obs, eqw_obs_err, addl_wave=[], addl_flux=[], addl_flux_err=[])
        Returns the Bayesian P(LAE)/P(OII) from EW distributions of LAEs and OII galaxies from
        Leung+ 2017
        
        Maximum P(LAE)/P(OII) is capped at 999. There is no minimum value but 0.0 represents an
        error or inability to calcluate the ratio.
        
        Uses ELiXer standard options and cosmology
        
        :param line_wave:  observed wavelength of the emission line in angstroms
        :param line_flux:  observed line flux (cgs)
        :param line_flux_err: observed line flux error (cgs)
        :param eqw_obs: observed equivalent width
        :param eqw_obs_err: observed equivalent width error
        :param addl_wave: array of additional emission line wavelengths in angstroms
        :param a

wave = 3711.0
flux = 1.62e-15
flux_err = 4.0e-17
ew = 500.0
ew_err = 60.0
addl_wave = []
addl_flux = []
addl_flux_err = []
classify.plae_poii(wave,flux,flux_err,ew,ew_err,None,None,None)

# Example call

In this case, this strongly favors LyA over OII (in point of fact, given the wavelength, OII is not possible).

In [3]:
wave = 3711.0 
flux = 1.7e-15 
flux_err = 4.0e-17 
ew = 600.0 
ew_err = 0.0 

classify.plae_poii(wave,flux,flux_err,ew,ew_err,None,None,None)


(999, 1.0, 9.999999999999999e-33)

# Using mostly the same data, we place the line at a longer wavelength where OII is possible and reduce the equivalent width

Now, this strongly suggests OII is far more likely than LyA

In [12]:
wave = 4711.0 
flux = 1.7e-15 
flux_err = 4.0e-17 
ew = 300.0 
ew_err = 0.0 

classify.plae_poii(wave,flux,flux_err,ew,ew_err,None,None,None)

(0.0528704691602368, 0.05021554949908128, 0.9497844505009188)